### 이전 대화를 기억하는 Chain 생성방법

in_memory를 가볍게 사용하는 방법이다. 

Runnable history를 사용하는, 영구적인 저장 방법이 아닌, 임시적인 저장방법이다.

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
from langchain_teddynote import logging

logging.langsmith("CH05-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Memory


### 이전 대화 내용을 기억하는 multi-turn Chain

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_community.chat_message_histories import ChatMessageHistory
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system', 
            "당신은 Question-Answering Chatbot입니다. 주어진 질문에 대하여 답변을 제공해 주세요."
        ),
        MessagesPlaceholder(variable_name='chat_history'), # 대화내용이 쌓이는 곳.
        (
            'human',
            '#Question:\n{question}'
        )
    ]
)
# prompt의 변수는 question과 chat_history 2개다.

llm = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | llm | StrOutputParser()

대화를 기록하는 체인 `chain_with_history`

In [4]:
# 세션을 저장하는 딕셔너리
store = {}

def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

In [15]:
# prompt의 question과 chat_history 확인
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history, # 세션 기록을 가져오는 함수
    input_messages_key="question", # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history" # 기록 메세지의 키
)

In [16]:
chain_with_history.invoke(
    {"question": "나의 이름은 재호입니다."},
    config={"configurable": {"session_id": "abc123"}}
)

[대화 세션ID]: abc123


'안녕하세요, 재호님! 만나서 반갑습니다. 무엇을 도와드릴까요?'

In [17]:
chain_with_history.invoke(
    {"question": "내 이름이 뭐라고??"},
    config={"configurable": {"session_id": "abc123"}}
)

[대화 세션ID]: abc123


'재호님이라고 하셨습니다!'

In [18]:
chain_with_history.invoke(
    {"question": "내 이름이 뭐라고??"},
    config={"configurable": {"session_id": "abc1234"}}
)

[대화 세션ID]: abc1234


'죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 좋겠습니다!'

: 